## Mount GDrive for Collab

In [ ]:
print('Mounting Gdrive')
from google.colab import drive
drive.mount('/content/drive')

import sys
import os
sys.path.append('/content/drive/MyDrive/')

## Lib

In [1]:
import os
import tensorflow as tf

from mlflow import MlflowClient
import mlflow.tensorflow

2024-04-23 16:02:18.243101: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-23 16:02:18.275325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-23 16:02:18.779889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Vars

In [2]:
model_path = "../../storage/models/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"
epochs = 5
learning_rate = 1e-4

In [8]:
train_dataset_path = "../../storage/datas/tf_datasets/train_dataset"
test_dataset_path = "../../storage/datas/tf_datasets/test_dataset"
val_dataset_path = "../../storage/datas/tf_datasets/val_dataset"
tracking_uri = "../../storage/mlruns"
model_checkpoint_url = "../../storage/model_checkpoint"
model_path = "../../storage/models/generated_model_efficientnetv2_21k_finetuned_1k_v1.keras"

## Load model & Datasets

### Model

In [9]:
# Load Model
import keras
model = keras.saving.load_model(model_path)

2024-04-23 16:04:04.164536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-23 16:04:04.188636: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [10]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_18 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetv2-b0 (Functional)  │ (None, 7, 7, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 11)             │         2,827 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,709,339 (25.59 MB)

 Trainable params: 790,027 (3.01 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [17]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

### Datasets

In [12]:
# Load datasets
train_datas = tf.data.experimental.load(train_dataset_path)
val_datas = tf.data.experimental.load(val_dataset_path)

Instructions for updating:
Use `tf.data.Dataset.load(...)` instead.


## Fit

In [24]:
# Callbacks
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(model_checkpoint_url, "model_{epoch:02d}_{val_loss:.2f}.weights.h5"),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=3,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=5)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.0001,
    cooldown=0)

### Mlflow Client


In [14]:
# Start mlflow client
mlflow.set_tracking_uri(tracking_uri)
client = MlflowClient()
mlflow.tensorflow.autolog()

In [ ]:
history = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

In [ ]:
history2 = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs+5,
    initial_epoch = 5,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

In [27]:
# Fine tuning
model.layers[1].trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-6),
              loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=3)])

In [ ]:
history3 = model.fit(
    train_datas,
    validation_data = val_datas,
    epochs = epochs,
    initial_epoch = 0,
    callbacks=[model_checkpoint, early_stopping, reduce_lr],
    verbose = 1)

### Save Model

In [ ]:
# Save model
model_path = "../../storage/models/generated_model_efficientnetv2_21k_finetuned_1k_v1_trained.keras"
model.save(filepath = model_path, overwrite = True)